<a href="https://colab.research.google.com/github/hyuna0926/cp2_phase2/blob/main/class_recommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 필요 라이브러리 및 데이터 로드

In [1]:
! pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 42.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from scipy import sparse
from scipy.sparse import csr_matrix
from tqdm.notebook import tqdm
import numpy as np
from datetime import datetime

import warnings
# 경고 제거
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from implicit.als import AlternatingLeastSquares as ALS
import implicit
import random
import os

In [3]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [4]:
path = '/content/drive/MyDrive/CP2_Phase2/'
'''
cart_purchase: 구매이력
product : 상품 정보(31121)
customer_info : 회원 정보(0~100,000)
'''
cart_purchase = pd.read_parquet(path + 'cart_purchase.parquet')
product = pd.read_parquet(path + 'product.parquet')
customer_info = pd.read_parquet(path + 'customer_info.parquet')

# 데이터 만들기

In [5]:
class Data:
  '''
  c_idx를 입력했을 때 원하는 데이터 나올 수 있게 하기?~!
  1. 아예 없는 경우
  2. 구매하지 않은 회원(customer_info.parquet)
  3. 20개 미만 산 회원(cart_purchase)
  4. 20개 이상 산 회원(cart_purchase)
  '''
  def __init__(self):
    pass


  def customer_info(self, c_idx):
    # 비회원
    if c_idx not in customer_info['c_idx']:
      print(c_idx, '님은 회원정보가 없습니다.')

    # 구매하지 않는 회원
    elif c_idx not in cart_purchase['c_idx']:
      print(c_idx, '님은 구매이력이 없습니다.')

    # 20개 이상 산 회원
    elif c_idx in self.customer_data()[2]['c_idx'].values: # upper
      print(c_idx, '님은 20개 이상 구매한 회원입니다. ALS 진행')
      
    elif c_idx in self.customer_data()[3]['c_idx'].values: #lower
      print(c_idx, '님은 20개 미만 구매한 회원입니다. CB 진행')

    return c_idx

  
  def customer_data(self):
    '''
    ALS와 CB 데이터 만들기

    return 
    train, test : ALS
    upper, lower : CB(20개 기준으로 나눔)
    '''
    df = cart_purchase.copy()
    df['values'] = 1  # 구매, 장바구니니까 1로 implicit 데이터

    df_group = df.groupby(['c_idx'], as_index=False).count()


    # ALS(20개 이상)
    upper = df[df['c_idx'].isin(df_group.query('values>=20').c_idx)] # 20개 이상
    test = upper.groupby('c_idx').sample(frac=0.2, random_state=42) # als_test
    train = upper.drop(test.index) # als_train
    
    # CB(20개 미만)
    lower = df[df['c_idx'].isin(df_group.query('values<20').c_idx)] # 20개 미만

    return train, test, upper, lower
  


  def no_purchase_data(self):
    '''
    회원정보는 있지만 구매하지 않은 사람을 위한 나이별 데이터 만들기
    customer_info와 cart_purchase 합치기

    return
    youth : 20대 이하 구매 이력
    age_20 : 20대 구매 이력
    age_30 : 30대 이상 구매 이력
    '''
    customer_c = customer_info.copy()
    customer_c.birthdate = customer_c.birthdate.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
    customer_c['age']=customer_c.birthdate.apply(lambda x: datetime.today().year - x.year)

    cus_pur = pd.merge(cart_purchase, customer_c, on=['c_idx','customer_id'], how='left')

    #데이터 분리
    youth = cus_pur.query("age<20")  # 20대 이하 구매 이력
    age_20 = cus_pur.query('age>=20 and age<30')  # 20대 구매 이력
    age_30 = cus_pur.query('age>=30')  # 30대 이상 구매 이력

    return youth, age_20, age_30



In [ ]:
data = Data()
train, test, upper, lower = data.customer_data() 

In [ ]:
data.customer_info(6934)

6934 님은 20개 미만 구매한 회원입니다. CB 진행


6934

In [ ]:
train, test, upper, lower = data.customer_data() # 데이터 잘 나눠짐

In [ ]:
youth, age_20, age_30 = data.no_purchase_data() # 데이터 잘 나눠짐

# CB

In [6]:
class CB_recommend():
  '''
  20개 미만 구매한 고객에게 성별별로 추천하기
  tf-idf를 이용한 컨텐츠 기반 추천시스템
  '''
  def __init__(self):
    pass
  
  def CB_data(self):
    '''
    CB data 만들어주기
    성별 나눠줌
    - 남자 ['Men','Boys','Unisex'] 17712개
    - 여자 ['Women','Girls','Unisex'] 15157개
    - Unisex는 모두 포함

    return 
    Men : 남성 상품 정보
    Women : 여성 상품 정보
    product_c : 중복값을 제거한 전체 상품 정보
    '''

    # product 중복값 제거
    product_c = product.drop_duplicates(subset=['productDisplayName'], keep='first',ignore_index=True)
    # 결측값 제거
    df = product_c.copy()
    df.dropna(subset=['productDisplayName','year'],inplace=True)
    col = ['baseColour','season','usage']
    df[col]=df[col].astype('object')
    df[col] = df[col].fillna("unknown") #결측값 채워주기
    df[col]=df[col].astype('category')
    
    #tf-idf를 위해 컬럼 만들어주기
    df['features'] = df[['gender','articleType','baseColour','season','usage']].apply(' '.join, axis=1)

    # 성별 나눠주기
    men = ['Men','Boys','Unisex']
    women = ['Women','Girls','Unisex']
    Men = df[df['gender'].isin(men)]
    Women = df[df['gender'].isin(women)]

    return Men, Women ,product_c

  def tfidf(self):
    '''
    tf-idf을 이용한 컨텐츠 기반 추천시스템
    남녀별로 코사인 유사도 진행 
    
    return cosine_men, cosine_women
    '''
    Men, Women, product_c = self.CB_data()

    tfidf_m = TfidfVectorizer()
    tfidf_men = tfidf_m.fit_transform(Men['features'])
    tfidf_w = TfidfVectorizer()
    tfidf_women = tfidf_w.fit_transform(Women['features'])


    #코사인 유사도
    cosine_men = pd.DataFrame(cosine_similarity(tfidf_men,tfidf_men),index = Men.p_idx, columns=Men.p_idx)
    cosine_women = pd.DataFrame(cosine_similarity(tfidf_women,tfidf_women),index = Women.p_idx, columns=Women.p_idx)

    return cosine_men, cosine_women


  def recommend(self, c_idx, k=25):
    '''
    Data클래스에 있는 데이터 들고오기
    남자/여자 나눠서 추천
    최근에 산 제품과 유사한 상품 k개 추천

    return recommend(유사한 상품 k개)
    '''
    data = Data()
    lower = data.customer_data()[3] # 20개 이하 구매한 사람

    Men, Women, product_c = self.CB_data()
    cosine_men, cosine_women = self.tfidf()

    # 고객이 최근에 산 상품
    buy_recent = lower[lower['c_idx']==c_idx].sort_values(by='created_at', ascending=False)[:1]  
    p_idx = buy_recent.p_idx.values[0]


    if p_idx in Men['p_idx']: #상품 p_idx가 Men에 들어가있으면
      men_sim = cosine_men[p_idx].sort_values(ascending=False).index
      recommend = product_c[product_c['p_idx'].isin(men_sim)][:k]
    
    else: # Women에 들어가있으면
      women_sim = cosine_women[p_idx].sort_values(ascending=False).index
      recommend = product_c[product_c['p_idx'].isin(women_sim)][:k]

    return recommend
    
   

In [ ]:
cb_recommend = CB_recommend()
cb_recommend.recommend(6934)

,product_id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,p_idx
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,27781
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,20698
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants,16433
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,22200
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011,Casual,Inkfruit Mens Chain Reaction T-shirt,12874
6,30805,Men,Apparel,Topwear,Shirts,Green,Summer,2012,Ethnic,Fabindia Men Striped Green Shirt,8473
8,29114,Men,Accessories,Socks,Socks,Navy Blue,Summer,2012,Casual,Puma Men Pack of 3 Socks,22387
9,30039,Men,Accessories,Watches,Watches,Black,Winter,2016,Casual,Skagen Men Black Watch,26021
10,9204,Men,Footwear,Shoes,Casual Shoes,Black,Summer,2011,Casual,Puma Men Future Cat Remix SF Black Casual Shoes,22136
12,18653,Men,Footwear,Flip Flops,Flip Flops,Black,Fall,2011,Casual,Fila Men Cush Flex Black Slippers,9695


# Nobuy_recommend

In [7]:
class Nobuy_recommend:
  def __init__(self):
    pass
  
  def nobuy_recommend(self,c_idx, k=25):
    '''
    아무 정보 없는 사람과 구매 이력이 없는 사람에게 추천
    '''
    data = Data()
    youth, age_20, age_30 = data.no_purchase_data()
    product_c = product.drop_duplicates(subset=['productDisplayName'], keep='first',ignore_index=True)

    if c_idx not in customer_info['c_idx'].values:  # 회원정보가 없는
      item = cart_purchase.groupby('p_idx').count()['created_at'].sort_values().tail(k).index
      recommend = product_c[product_c['p_idx'].isin(item)]
      print('회원정보가 없어 가장 많이 팔린 제품을 추천합니다')

    elif c_idx in youth['c_idx'].values: #20세 미만
      item = youth.groupby('p_idx').count().sort_values(by='created_at').tail(k).index
      recommend= product_c[product_c['p_idx'].isin(item)]
      print('회원님! 20대 미만인 고객님들이 가장 많이 구매한 상품을 추천합니다.')
  
    elif c_idx in age_20['c_idx'].values: #20대
      item = age_20.groupby('p_idx').count().sort_values(by='created_at').tail(k).index
      recommend= product_c[product_c['p_idx'].isin(item)]
      print('20대 회원님! 20대인 고객님들이 가장 많이 구매한 상품들을 추천합니다.')
    
    else: #30대 이상
      item = age_30.groupby('p_idx').count().sort_values(by='created_at').tail(k).index
      recommend= product_c[product_c['p_idx'].isin(item)]
      print('30대 이상인 회원님! 30대 이상인 고객님들이 가장 많이 구매한 상품들을 추천합니다.')

    return recommend


In [ ]:
nobuy = Nobuy_recommend()

In [ ]:
nobuy.nobuy_recommend(60000)

30대 이상인 회원님! 30대가 가장 많이 구매한 상품들을 추천합니다.


,product_id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,p_idx
54,59435,Men,Footwear,Shoes,Formal Shoes,Black,Summer,2012,Formal,Arrow Men Black Formal Shoes,2580
76,54118,Women,Footwear,Shoes,Heels,Black,Winter,2012,Casual,Rocia Women Black Flats,25016
123,49005,Women,Accessories,Jewellery,Ring,Silver,Summer,2012,Casual,Lucera Women Silver Ring,16319
810,47191,Men,Footwear,Shoes,Formal Shoes,Black,Summer,2012,Formal,Franco Leone Men Black Formal Shoes,10691
948,7390,Men,Footwear,Shoes,Formal Shoes,Brown,Winter,2018,Formal,Red Tape Men Brown Shoes,24194
1110,3398,Women,Accessories,Bags,Handbags,Black,Winter,2015,Casual,Murcia Women Black Handbag,17273
1136,45407,Women,Footwear,Shoes,Flats,Brown,Summer,2012,Casual,Rocia Women Brown Sandals,25049
1249,30692,Women,Footwear,Shoes,Heels,Brown,Winter,2015,Casual,Catwalk Women Brown Heels,5280
1370,48728,Women,Accessories,Jewellery,Earrings,Silver,Summer,2015,Casual,Lucera Women Silver Earrings,16314
1424,47794,Women,Accessories,Bags,Handbags,Brown,Summer,2012,Casual,Murcia Women Brown Handbag,17291


# ALS(implicit 라이브러리)

In [8]:
from inspect import modulesbyfile
class ALS_library:
  '''
  implicit 라이브러리를 이용한 ALS 모델
  '''

  def __init__(self,model):
    self.model=model
  
  def als_data(self):
    data = Data()
    train, test, upper, lower = data.customer_data()
    
    # csr matrix 만들기
    csr_train = sparse.csr_matrix((train['values'],(train['c_idx'], train['p_idx'])))
    csr_test = sparse.csr_matrix((test['values'],(test['c_idx'], test['p_idx'])))

    # 검증셋 만들기
    test_df = test.groupby('c_idx')['p_idx'].unique().to_frame().reset_index()

    #중복값 제거
    product_c = product.drop_duplicates(subset=['productDisplayName'], keep='first',ignore_index=True)
    
    return csr_train, csr_test, test_df, product_c  # 0,1,2,3


  def fit(self):
    # als 학습
    self.model.fit(self.als_data()[0])


  def recommendation(self, c_idx, k=25):
    # 추천하기  
    csr_train, csr_test, test_df, product_c = self.als_data()
    item = self.model.recommend(c_idx, csr_train[c_idx], k)[0]
    recommend = product_c[product_c['p_idx'].isin(item)]

    return recommend
  

  def mean_precisin_hit(self, user,k=25):
    csr_train, csr_test, test_df, product_c = self.als_data()

    hit = 0
    precision = 0
    users = 0
    for c_idx in range(user):
      if c_idx in test_df['c_idx'].values:
        hit_count = 0  # for문 돌 때마다 리셋
        users += 1
        recommend = self.model.recommend(c_idx, csr_train[c_idx],k)[0]
        buy_test = test_df[test_df['c_idx']==c_idx].p_idx.values[0]

        for i in buy_test:
          for j in recommend:
            if i==j:
              hit_count+=1
              precision+=1

        if hit_count >= 1: # count가 1 이상이면 hit한 것이니까 전체에 1 추가
          hit+=1

    hit_rate = hit/users
    mean_precision = (precision/25)/users

    return hit_rate, mean_precision




In [ ]:
factor=180;alpha=1;regularization=32;iteration=2
ALS = implicit.als.AlternatingLeastSquares(factors=factor, alpha=alpha, regularization=regularization,
                                            iterations=iteration, random_state=42,calculate_training_loss=True)

In [ ]:
als = ALS_library(ALS)
als.fit()

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
als.recommendation(800,25)

,product_id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,p_idx
54,59435,Men,Footwear,Shoes,Formal Shoes,Black,Summer,2012,Formal,Arrow Men Black Formal Shoes,2580
76,54118,Women,Footwear,Shoes,Heels,Black,Winter,2012,Casual,Rocia Women Black Flats,25016
810,47191,Men,Footwear,Shoes,Formal Shoes,Black,Summer,2012,Formal,Franco Leone Men Black Formal Shoes,10691
948,7390,Men,Footwear,Shoes,Formal Shoes,Brown,Winter,2018,Formal,Red Tape Men Brown Shoes,24194
1110,3398,Women,Accessories,Bags,Handbags,Black,Winter,2015,Casual,Murcia Women Black Handbag,17273
1136,45407,Women,Footwear,Shoes,Flats,Brown,Summer,2012,Casual,Rocia Women Brown Sandals,25049
1249,30692,Women,Footwear,Shoes,Heels,Brown,Winter,2015,Casual,Catwalk Women Brown Heels,5280
1370,48728,Women,Accessories,Jewellery,Earrings,Silver,Summer,2015,Casual,Lucera Women Silver Earrings,16314
1424,47794,Women,Accessories,Bags,Handbags,Brown,Summer,2012,Casual,Murcia Women Brown Handbag,17291
1497,55635,Women,Footwear,Shoes,Heels,Gold,Winter,2015,Casual,Catwalk Women Gold Heels,5355


In [ ]:
als.mean_precisin_hit(100)

(0.29411764705882354, 0.01568627450980392)

# tunning


In [14]:
class Tunning:
  def __init__(self):
    pass
  
  def random_search(self,params, n_iters=10):
      results =[]
      for _ in tqdm(range(n_iters)):
        factor = np.random.choice(params['factor'])
        alpha = np.random.choice(params['alpha'])
        regularization = np.random.choice(params['regularization'])
        iteration = np.random.choice(params['iteration'])

        ALS = implicit.als.AlternatingLeastSquares(factors=factor, alpha=alpha, regularization=regularization, iterations=iteration,
                  random_state=42, calculate_training_loss=True)

        als = ALS_library(ALS)
        als.fit()
        hit_rate, mean_precision = als.mean_precisin_hit(5000)

        
        result = [factor, alpha, regularization, iteration, hit_rate, mean_precision]
        results.append(result)
      frame = pd.DataFrame(results, columns=['factor','alpha','regularization','iteration','hit_rate','mean_precision'])
      frame = frame.sort_values('hit_rate', ascending=False)

      return frame
  
  def grid_search(self, params):
    results =[]
    for factor in tqdm(params['factor']):
      for alpha in params['alpha']:
        for regularization in params['regularization']:
          ALS = implicit.als.AlternatingLeastSquares(factors=factor, alpha=alpha, regularization=regularization, iterations=20,
                random_state=42, calculate_training_loss=True)
          als = ALS_library(ALS)
          als.fit()
          hit_rate, mean_precision = als.mean_precisin_hit(5000)

          
          result = [factor, alpha, regularization, iteration, hit_rate, mean_precision]
          results.append(result)
        frame = pd.DataFrame(results, columns=['factor','alpha','regularization','iteration','hit_rate','mean_precision'])
        frame = frame.sort_values('hit_rate', ascending=False)
    
    return frame

In [ ]:
params_grid = {
    'factor': [160,180,200],
    'alpha' : [1,3,5],
    'regularization' : [28,30,32],
    'iteration' : np.arange(15,31,5)
}
tunner = Tunning()
tunner.grid_search(params_grid)

In [15]:
params_random = {
    'factor': np.arange(90,210,10),
    'alpha' : np.arange(1,21,4),
    'regularization' : np.arange(1,30,5),
    'iteration' : np.arange(15,31,5)
}
tunner = Tunning()
tunner.random_search(params_random,n_iters=3)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

,factor,alpha,regularization,iteration,hit_rate,mean_precision
2,160,5,21,20,0.168237,0.008057
1,120,9,11,15,0.159713,0.007609
0,160,13,6,25,0.152086,0.007250
